In [44]:
from sklearn import preprocessing
from sklearn.decomposition import PCA 
import numpy as np
import pandas as pd 
import math
import time 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

np.random.seed(7)

output_dim = 5

# Data loading
df_lp1 = 'lp1_data.csv'
df_lp2 = 'lp2_data.csv'
df_lp3 = 'lp3_data.csv'
df_lp4 = 'lp4_data.csv'
df_lp5 = 'lp5_data.csv'
dfs = [df_lp1, df_lp2, df_lp3, df_lp4, df_lp5]



full_data = []
train_data = []
lp_tmp = []
for df in dfs:
    tmp = pd.read_csv(df)
    tmp = np.array(tmp)
    for tp in tmp:
        full_data.append(tp)

np.random.shuffle(full_data)
for tp in full_data:
    if tp[6] == 3:
        continue
    train_data.append(tp[0:6])
    if tp[6] == 3:
        lp_tmp.append(2)
    else :
        lp_tmp.append(tp[6])

# oneHot Encoding
train_label = np.zeros( ( len(train_data), output_dim ) )
for i in range(len(lp_tmp)):
    train_label[i][lp_tmp[i]-1] = 1     # let 1-5 turns to 0-4


In [45]:
#data preprocessing
x_scale = preprocessing.scale(train_data)
x_normalized = preprocessing.normalize(x_scale, norm='l2')
pca=PCA(n_components='mle', whiten=True)
newData=pca.fit_transform(x_normalized)

In [46]:
def initialize(Layer):
    Weights = []
    Bias = []
    for i in range ( 1, len(Layer) ):
        weight = np.random.randn( Layer[i-1], Layer[i]) * np.sqrt(2 / Layer[i])
        b = np.random.randn(Layer[i], 1) * np.sqrt(2 / Layer[i])
        Weights.append(weight)
        Bias.append(b)

    # to make index of layer 1 from 0 to 1
    Weights.insert(0, [])
    Bias.insert(0, [])
    Weights = np.asarray(Weights)
    Bias = np.asarray(Bias)

    return Weights, Bias

def Minmax_scale(w):
    return np.array( (w-w.min()) / (w.max()-w.min()) )
minmax = 0
def Partition(data, label, ratio):
    if minmax == 1 :
        data = np.transpose(data).astype(np.float64)
        for i in range (len(data) ):
            data[i] = Minmax_scale(data[i])
        data = np.transpose(data)
    t_data = data[0:int(len(data)*ratio) ]
    t_label = label[0:int(len(label)*ratio) ]
    verify_data = data[int(len(data)*ratio):len(data)]
    verify_label = label[int(len(data)*ratio):len(data)]
    return t_data, t_label, verify_data, verify_label

print("D")

def Sigmoid( n ):
    l = []
    for tmp in n:
        if np.sum(tmp) >= 0:
            l.append( 1 / (1 + math.exp(-tmp) ) )
        else:
            l.append( math.exp(tmp) / ( 1 + math.exp(tmp) )  )
    return l

# def sigmoid(data):
#   if np.sum(data) >= 0:
#     return  1 / ( 1 + math.exp(-np.sum(data)))
#   else:
#     return math.exp(np.sum(data)) / (1 + math.exp(np.sum(data)) )

def Binary_crossEntropy(y, a):
    CE = 0
    for i in range( len(y) ):
        CE += y[i] * math.log(a[i] + 1e-15 ) + (1 - y[i]) * math.log(1 - a[i] + 1e-15)
    return CE
def FeedForward(data, Weight, bias):
    n = np.matmul( data, Weight) + bias.transpose() 
    return np.asarray( Sigmoid(n[0]) ) 

def Prediction(a):
    index = 0
    for i in range( len(a) ):
        if a[i] > a[index]:
            index = i
    return index

D


In [47]:
train_ratio = 0.8

Layer = [16, 8, 6, 5]


Layer.insert(0, len(newData[0]))
Layer.append(output_dim)

Weights, Bias = initialize(Layer)
t_data, t_label, verify_data, verify_label = Partition(newData, train_label, train_ratio)

C:\Users\ggg59\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [48]:
learning_rate = 0.01
Epoch = 5001
tau = 0.001

end_flag = False
last_acc_T = 0
last_acc_V = 0
ce = []
writee = False
strr = ""
for ep in range(Epoch):
    totalLoss = 0
    L = len(Layer)-1
    acc_t = 0
    acc_v = 0
    flag = (ep % 10 == 0)
    string = ""
    for i in range(len(t_data)):
        a = []
        a.append( t_data[i] )
        for l in range(1, L+1 ):
            a.append(FeedForward(a[l-1],  Weights[l], Bias[l]) )
        
        # BackWard
        Delta = [ a[L] - t_label[i] ]
        for l in range(L-1, 0, -1):
            Delta.insert(0, np.matmul( Delta[0], Weights[l+1].transpose() ) * ( a[l]*(1-a[l])  )  )
        Delta.insert(0, [])
        
        # update Weights and Bias
        for l in range(1, L+1):
            t = np.matmul( Delta[l][np.newaxis].T, a[l-1][np.newaxis] )
            Weights[l] = Weights[l] - learning_rate * t.T
            Bias[l] = Bias[l] - learning_rate * Delta[l]
        
        # Calculate loss and training accuracy
        if flag : 
            totalLoss -= Binary_crossEntropy( t_label[i], a[L] )
            p = Prediction(a[L])
            if t_label[i][p] == 1:
                acc_t += 1

    # Calculate verify accuracy
    if flag : 
        for i in range( len(verify_data) ):
            out = verify_data[i]
            for l in range(1, L+1 ):
                out = FeedForward(out, Weights[l], Bias[l])
            p = Prediction(out)
            if verify_label[i][p] == 1:
                acc_v += 1

        if last_acc_T < acc_t / len(t_data) and last_acc_V > acc_v / len(verify_data):
            print("< Overtraining leads to overfitting >" )
            end_flag = True
        last_acc_T = acc_t
        last_acc_V = acc_v

    if (ep+1) == Epoch:
        print("< Reach maximum Epoch >" )
        end_flag = True

    if flag and totalLoss / len(t_data) < tau:
        print("< Loss is low enough ( tau: " + str(tau) + ") >"  )
        end_flag = True

    if flag:
        print("<<< Status" + " " + strr + " >>>")
        print("Number of train_data: " + str( len(t_data)) )
        print("Number of verify_data: " + str( len(verify_data)) )
        print("Number of Hidden Layer: " + str( len(Layer)-2) )
        print("Number of Neuron in each Hidden Layer: " + str(Layer[1:len(Layer)-1]) )
        print("Learning Rate: " + str(learning_rate) )
        print("Epoch: " + str(ep+1) )
        print("Loss: " + str( totalLoss / len(t_data) ))
        print("Train accuracy: " + str(acc_t / len(t_data)) )
        print("Verify accuracy: " + str(acc_v / len(verify_data)) )
        
    if writee :    
        with open('Results\\' + strr + ' ' + 'Result '+time.strftime('%Y-%m-%d %H-%M-%S', time.localtime()) +  '.txt', mode = 'w') as text_file:
            string += "<<<Status>>>\n"
            string += "Number of train_data: " + str( len(t_data)) +"\n"
            string += "Number of verify_data: " + str( len(verify_data)) +"\n"
            string += "Number of Hidden Layer: " + str( len(Layer)-2) +"\n"
            string += "Number of Neuron in each Hidden Layer: " + str(Layer[1:len(Layer)-1]) +"\n"
            string += "Learning Rate: " + str(learning_rate) +"\n"
            string += "Epoch: " + str(ep+1) +"\n"
            string += "Loss: " + str( totalLoss / len(t_data) )+"\n"
            string += "Train accuracy: " + str(acc_t / len(t_data)) +"\n"
            string += "Verify accuracy: " + str(acc_v / len(verify_data)) +"\n"
            text_file.write(string)
            text_file.close()

    if end_flag:
        break

0.5795272435897436
Verify accuracy: 0.5216346153846154
<<< Status  >>>
Number of train_data: 4992
Number of verify_data: 1248
Number of Hidden Layer: 4
Number of Neuron in each Hidden Layer: [16, 8, 6, 5]
Learning Rate: 0.01
Epoch: 4291
Loss: 1.2847705008288843
Train accuracy: 0.5805288461538461
Verify accuracy: 0.5224358974358975
<<< Status  >>>
Number of train_data: 4992
Number of verify_data: 1248
Number of Hidden Layer: 4
Number of Neuron in each Hidden Layer: [16, 8, 6, 5]
Learning Rate: 0.01
Epoch: 4301
Loss: 1.2857649857524411
Train accuracy: 0.5809294871794872
Verify accuracy: 0.5288461538461539
<<< Status  >>>
Number of train_data: 4992
Number of verify_data: 1248
Number of Hidden Layer: 4
Number of Neuron in each Hidden Layer: [16, 8, 6, 5]
Learning Rate: 0.01
Epoch: 4311
Loss: 1.2917636949152655
Train accuracy: 0.5803285256410257
Verify accuracy: 0.530448717948718
<<< Status  >>>
Number of train_data: 4992
Number of verify_data: 1248
Number of Hidden Layer: 4
Number of Neuro

In [49]:
class MLP(nn.Module):
    def __init__(self, input, output):
        super(MLP,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input, 8),
            nn.BatchNorm1d(8),
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16, 16),
            nn.ReLU()
        )
        self.layer4 = nn.Sequential(
            nn.Linear(16, 8),
            nn.ReLU()
        )
        self.layer5 = nn.Sequential(
            nn.Linear(8, output),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        return x
model = MLP(len(t_data[0]), output_dim)
criterion = nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


for epoch in range(Epoch):
    acc_t = 0
    mean_loss = 0
    optimizer.zero_grad()
    inputs = torch.autograd.Variable( torch.FloatTensor(t_data))
    outputs = model(inputs)
    loss = criterion( outputs, torch.tensor(t_label) )
    mean_loss += loss.item()
    loss.backward()
    optimizer.step()
    #print( mean_loss/len(t_data) )
    if epoch % 100 == 0:
        for i in range (len(outputs)):
            p = Prediction(outputs[i])
            if t_label[i][p] == 1:
                acc_t += 1
        print("acc: " + str(acc_t / len(t_data)))

acc: 0.046875
acc: 0.5274439102564102
acc: 0.5412660256410257
acc: 0.5466746794871795
acc: 0.5464743589743589
acc: 0.546073717948718
acc: 0.5502804487179487
acc: 0.5490785256410257
acc: 0.5472756410256411
acc: 0.5510817307692307
acc: 0.5518830128205128
acc: 0.5540865384615384
acc: 0.5528846153846154
acc: 0.5552884615384616
acc: 0.553886217948718
acc: 0.5568910256410257
acc: 0.5570913461538461
acc: 0.5424679487179487
acc: 0.5570913461538461
acc: 0.5586939102564102
acc: 0.5584935897435898
acc: 0.5588942307692307
acc: 0.5596955128205128
acc: 0.5608974358974359
acc: 0.5470753205128205
acc: 0.5598958333333334
acc: 0.5582932692307693
acc: 0.561698717948718
acc: 0.5600961538461539
acc: 0.5610977564102564
acc: 0.561698717948718
acc: 0.5629006410256411
acc: 0.561698717948718
acc: 0.561698717948718
acc: 0.5614983974358975
acc: 0.561698717948718
acc: 0.5578926282051282
acc: 0.5625
acc: 0.5618990384615384
acc: 0.5629006410256411
acc: 0.5584935897435898
acc: 0.5627003205128205
acc: 0.56109775641025

In [50]:
outputs

tensor([[0.0000e+00, 3.7424e-23, 3.2497e-25, 4.3980e-01, 5.6020e-01],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00],
        ...,
        [6.3819e-29, 1.5749e-28, 3.5937e-35, 0.0000e+00, 1.0000e+00],
        [1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.5303e-22],
        [1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 8.9771e-17]],
       grad_fn=<SoftmaxBackward>)